In [1]:
import time
from pathlib import Path
from wt_tools.replays_db.replays_db import ReplaysDb, br_from, er_from
from collections import Counter
from operator import itemgetter

get_info = itemgetter('info')
get_rank = itemgetter('rank')
get_units = itemgetter('units')

replays_path = Path('/home/kotiq/games/WarThunder/Replays')
db_path = Path("/tmp/query/replays.d")
db = ReplaysDb(replays_path=replays_path, db_path=db_path)
t0 = time.time()
db.update_from_replays()
t1 = time.time()
print(f'merge dt = {t1 - t0:.2f}')
print(db.summary())

merge dt = 9.26
len replays = 104
len users = 3035
len units = 455


In [2]:
users = db.tables['users']
units = db.tables['units']
replays = db.tables['replays']

Участники сессий:

In [3]:
min_uid = min(users)
max_uid = max(users)
print(f'uids: [{min_uid:_}..{max_uid:_}]')

uids: [162_017..125_020_060]


Экономический ранг и боевой рейтинг машины:

In [4]:
brummbar_name = 'germ_sturmpanzer_IV_brummbar'
brummbar_rank = units[brummbar_name]['rank']
brummbar_br = br_from(brummbar_rank)
print(f'Brummbar\'s (rank, br): ({brummbar_rank}, {brummbar_br})')

Brummbar's (rank, br): (10, 4.3)


Номер и ник игрока:

In [5]:
my_uid = None
my_nick = 'kotiq'
for uid_, nick_ in users.items():
    if my_nick == nick_:
        my_uid = uid_
        break
print(f'Playser\'s (uid, nick): ({my_uid}, {my_nick})')            

Playser's (uid, nick): (38004121, kotiq)


In [6]:
def part_by(replays, update, uid, unit, rank):
    counter = Counter()
    for info in map(get_info, replays.values()):
        data = info.get(uid)
        if data and unit in get_units(data) and rank == get_rank(data): 
            update(counter, info)
    return counter

Часть боев с разницей ранга игрока и наибольшего ранга в этом бою:

In [7]:
def update_max_rank(counter, info):
    max_rank = max(map(get_rank, info.values()))
    counter.update([max_rank])

counter = part_by(replays, update_max_rank, my_uid, brummbar_name, brummbar_rank)
total = sum(counter.values())
print(f'total sessions: {total}')
for rank in sorted(counter):    
    count = counter[rank]
    print(f'{rank - brummbar_rank} => {100*count/total:>4.1f}%')

total sessions: 52
0 => 17.3%
1 => 17.3%
2 => 28.8%
3 => 36.5%


Часть боев с диапазонами разницы рангов игрока и остальных участников:

In [8]:
def update_min_max_rank(counter, info):
    max_rank = max(map(get_rank, info.values()))
    min_rank = min(map(get_rank, info.values()))
    counter.update([(min_rank, max_rank)])
        
counter = part_by(replays, update_min_max_rank, my_uid, brummbar_name, brummbar_rank)          
total = sum(counter.values())
print(f'total sessions: {total}')
print(f'target rank: {brummbar_rank}')
for p in sorted(counter):
    count = counter[p]
    rs = tuple(map(brummbar_rank.__rsub__, p))
    print(f'{rs!s:<7} => {100*count/total:>4.1f}%')

total sessions: 52
target rank: 10
(-3, 0) => 15.4%
(-2, 0) =>  1.9%
(-2, 1) => 15.4%
(-1, 1) =>  1.9%
(-1, 2) => 23.1%
(0, 2)  =>  5.8%
(0, 3)  => 36.5%
